In [35]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

In [36]:
# --- 0. FUNCIÓN PARA LIMPIAR EL TEXTO (NUEVA) ---
def clean_html(raw_html):
    if not raw_html:
        return ""
    
    # 1. Convertir entidades HTML (ej: &oacute; -> ó)
    text = html.unescape(raw_html)
    
    # 2. Reemplazar etiquetas HTML por espacios
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # 3. Eliminar espacios múltiples y saltos de línea
    text = " ".join(text.split())
    
    return text

In [37]:
# 1. Configuración y Carga
xml_file = "viaticos.bpmn"
# Manejo de error por si el archivo no existe al probar
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{xml_file}'. Asegúrate de que esté en la misma carpeta.")
    exit()

# Namespaces
ns = {
    'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL',
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}

# Lista final de datos
all_data = []

# --- Función recursiva modificada para aceptar 'descriptions' ---
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, process_name, order_counter):
    if node_id in visited:
        return order_counter
    
    visited.add(node_id)
    
    # Obtener datos del nodo
    name = elements.get(node_id, '')
    role = lanes.get(node_id, 'Sistema / General')
    e_type = element_types.get(node_id, 'Desconocido')
    
    # Obtener y limpiar la descripción (NUEVO)
    raw_desc = descriptions.get(node_id, '')
    clean_desc = clean_html(raw_desc)
    
    # Solo agregamos si tiene información relevante
    if name or e_type in ['Inicio', 'Fin', 'Compuerta']:
        clean_name = name.replace('\n', ' ').strip()
        
        all_data.append({
            'Proceso': process_name,
            'Orden': order_counter,
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,  # <--- CAMPO NUEVO
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Buscar siguientes pasos
    next_nodes = graph.get(node_id, [])
    
    for next_node in next_nodes:
        # Nota: pasamos 'descriptions' en la llamada recursiva
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, process_name, order_counter)
        
    return order_counter

In [38]:
# 2. Iterar sobre CADA PROCESO
for process in root.findall('bpmn:process', ns):
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Procesando: {process_name}...")

    # Diccionarios locales
    elements = {}      
    element_types = {} 
    lanes = {}
    descriptions = {} # <--- Diccionario para guardar descripciones
    graph = defaultdict(list)
    
    # A. Mapear Lanes (Roles)
    for lane_set in process.findall('bpmn:laneSet', ns):
        for lane in lane_set.findall('bpmn:lane', ns):
            lane_name = lane.get('name')
            for flow_node in lane.findall('bpmn:flowNodeRef', ns):
                lanes[flow_node.text] = lane_name

    # B. Extraer Nodos (Incluyendo lógica de documentación)
    
    # Helper interno para extraer info común (ID, Nombre, Doc)
    def extract_node_info(node, type_label):
        nid = node.get('id')
        elements[nid] = node.get('name', '')
        element_types[nid] = type_label
        
        # Buscar documentación dentro del nodo
        doc_node = node.find('bpmn:documentation', ns)
        if doc_node is not None and doc_node.text:
            descriptions[nid] = doc_node.text
        else:
            descriptions[nid] = ""

    # Tareas
    task_tags = ['task', 'userTask', 'serviceTask', 'sendTask', 'receiveTask', 'manualTask', 'businessRuleTask', 'scriptTask']
    for tag in task_tags:
        for task in process.findall(f'bpmn:{tag}', ns):
            extract_node_info(task, 'Tarea')

    # Eventos Inicio
    for start in process.findall('bpmn:startEvent', ns):
        extract_node_info(start, 'Inicio')

    # Eventos Fin
    for end in process.findall('bpmn:endEvent', ns):
        extract_node_info(end, 'Fin')
        
    # Eventos Intermedios
    for inter in process.findall('bpmn:intermediateCatchEvent', ns) + process.findall('bpmn:intermediateThrowEvent', ns):
        extract_node_info(inter, 'Evento')

    # Compuertas
    for gate in process.findall('bpmn:exclusiveGateway', ns) + process.findall('bpmn:parallelGateway', ns) + process.findall('bpmn:inclusiveGateway', ns):
        extract_node_info(gate, 'Compuerta')

    # C. Construir Conexiones
    for seq in process.findall('bpmn:sequenceFlow', ns):
        source = seq.get('sourceRef')
        target = seq.get('targetRef')
        if source and target:
            graph[source].append(target)

    # D. Ejecutar el Recorrido
    start_nodes = [nid for nid, type_ in element_types.items() if type_ == 'Inicio']
    
    visited = set()
    counter = 1
    
    if start_nodes:
        # Agregamos 'descriptions' a la llamada
        traverse_flow(start_nodes[0], visited, graph, elements, element_types, lanes, descriptions, process_name, counter)
    else:
        # Fallback para procesos sin evento de inicio formal
        all_nodes = set(elements.keys())
        targets = set([target for targets in graph.values() for target in targets])
        potential_starts = list(all_nodes - targets)
        if potential_starts:
             traverse_flow(potential_starts[0], visited, graph, elements, element_types, lanes, descriptions, process_name, counter)

Procesando: Proceso principal...
Procesando: Trámites y permisos ante el Ministerio de Defensa...


In [39]:
# 3. Exportar a CSV
if all_data:
    df = pd.DataFrame(all_data)

    # Ordenar columnas (incluyendo Descripción)
    cols = ['Proceso', 'Orden', 'Rol (Responsable)', 'Actividad', 'Descripción', 'Tipo', 'ID']
    df = df[cols]

    output_file = 'flujos_con_descripcion.csv'
    df.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')

    print("-" * 30)
    print(f"¡Listo! Se generó: {output_file}")
    print(df.head())
else:
    print("No se encontraron datos para exportar.")

------------------------------
¡Listo! Se generó: flujos_con_descripcion.csv
                                             Proceso  Orden  \
0  Trámites y permisos ante el Ministerio de Defensa      1   
1  Trámites y permisos ante el Ministerio de Defensa      2   
2  Trámites y permisos ante el Ministerio de Defensa      3   
3  Trámites y permisos ante el Ministerio de Defensa      4   
4  Trámites y permisos ante el Ministerio de Defensa      5   

   Rol (Responsable)                          Actividad  \
0  Sistema / General                                      
1  Sistema / General                   Tipo Colaborador   
2  Sistema / General                                      
3  Sistema / General  Recibir Certificación de tiquetes   
4  Sistema / General                                  2   

                                         Descripción       Tipo  \
0                                                        Inicio   
1                                                     C

In [40]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

# --- FUNCIÓN PARA LIMPIAR HTML ---
def clean_html(raw_html):
    if not raw_html: return ""
    text = html.unescape(raw_html)
    text = re.sub(r'<[^>]+>', ' ', text)
    return " ".join(text.split())

# 1. Carga del Archivo
xml_file = "viaticos.bpmn"
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró '{xml_file}'")
    exit()

# Namespaces genéricos para búsqueda
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

# --- PASO CRÍTICO 1: MAPA DE RECURSOS (ID -> NOMBRE) ---
resource_map = {}

# Buscamos <resource> en todo el archivo (sin importar dónde estén)
# Usamos iter() para recorrer todo el árbol recursivamente buscando la etiqueta 'resource'
for elem in root.iter():
    # Verificamos si la etiqueta termina en 'resource' (para ignorar namespaces)
    if elem.tag.endswith('resource'):
        r_id = elem.get('id')
        r_name = elem.get('name')
        if r_id and r_name:
            resource_map[r_id] = r_name

print(f"Recursos identificados en el archivo: {len(resource_map)}")
# Imprimimos 3 de ejemplo para verificar
print(f"Ejemplos de recursos: {list(resource_map.items())[:3]}")

# --- LISTA FINAL ---
all_data = []

# --- FUNCIÓN RECURSIVA ---
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter):
    if node_id in visited: return order_counter
    visited.add(node_id)
    
    # Datos básicos
    name = elements.get(node_id, '')
    e_type = element_types.get(node_id, 'Desconocido')
    clean_desc = clean_html(descriptions.get(node_id, ''))
    
    # --- DETERMINAR ROL (LÓGICA PRIORITARIA) ---
    # 1. ¿Tiene rol explícito asignado en la tarea?
    role = explicit_roles.get(node_id)
    
    # 2. Si no, ¿está en un carril (Lane)?
    if not role:
        role = lanes.get(node_id)
        
    # 3. Default
    if not role:
        role = 'Sistema / General'

    # Guardar si es relevante
    if name or e_type in ['Inicio', 'Fin', 'Compuerta']:
        clean_name = name.replace('\n', ' ').strip()
        all_data.append({
            'Proceso': process_name,
            'Orden': order_counter,
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Siguientes pasos
    for next_node in graph.get(node_id, []):
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter)
    
    return order_counter

# 2. PROCESAMIENTO
# Buscamos procesos ignorando namespace estricto para mayor compatibilidad
processes = [p for p in root.iter() if p.tag.endswith('process')]

for process in processes:
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Procesando flujo: {process_name}...")

    # Diccionarios locales
    elements = {}
    element_types = {}
    lanes = {}
    descriptions = {}
    explicit_roles = {}
    graph = defaultdict(list)
    
    # A. Lanes (Carriles)
    # Buscamos laneSets dentro del proceso actual
    for lane in process.iter():
        if lane.tag.endswith('lane'):
            lane_name = lane.get('name')
            # Buscar hijos flowNodeRef
            for ref in lane.iter():
                if ref.tag.endswith('flowNodeRef') and ref.text:
                    lanes[ref.text.strip()] = lane_name

    # B. Nodos y Roles Explícitos
    # Iteramos sobre TODOS los elementos dentro del proceso
    for node in process.iter():
        tag_clean = node.tag.split('}')[-1] # Quitamos namespace para leer fácil (ej: 'userTask')
        nid = node.get('id')
        
        if not nid: continue # Si no tiene ID, saltar

        # Tipificar
        if 'Task' in tag_clean: e_type = 'Tarea'
        elif 'Gateway' in tag_clean: e_type = 'Compuerta'
        elif 'startEvent' in tag_clean: e_type = 'Inicio'
        elif 'endEvent' in tag_clean: e_type = 'Fin'
        elif 'Event' in tag_clean: e_type = 'Evento'
        else: continue # Ignorar otros elementos de dibujo

        elements[nid] = node.get('name', '')
        element_types[nid] = e_type
        
        # --- EXTRACCIÓN DE DOCUMENTACIÓN ---
        for child in node:
            if child.tag.endswith('documentation') and child.text:
                descriptions[nid] = child.text

        # --- EXTRACCIÓN AGRESIVA DE ROLES ---
        # Si es una tarea, miramos dentro de TODOS sus hijos si aparece un ID de recurso
        if 'Task' in tag_clean:
            found_role = None
            # Iteramos recursivamente dentro de la tarea
            for subchild in node.iter():
                if subchild.text:
                    text_val = subchild.text.strip()
                    # A veces viene como "ns:ID", partimos por ":" y tomamos el último
                    candidate_id = text_val.split(':')[-1]
                    
                    if candidate_id in resource_map:
                        found_role = resource_map[candidate_id]
                        break # Encontramos uno, dejamos de buscar
            
            if found_role:
                explicit_roles[nid] = found_role

    # C. Conexiones (SequenceFlows)
    for seq in process.iter():
        if seq.tag.endswith('sequenceFlow'):
            source = seq.get('sourceRef')
            target = seq.get('targetRef')
            if source and target:
                graph[source].append(target)

    # D. Ejecutar DFS
    start_nodes = [n for n, t in element_types.items() if t == 'Inicio']
    visited = set()
    counter = 1
    
    if start_nodes:
        traverse_flow(start_nodes[0], visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, counter)
    else:
        # Fallback si no hay inicio claro
        all_ids = set(elements.keys())
        targets = set([x for sub in graph.values() for x in sub])
        roots = list(all_ids - targets)
        if roots:
            traverse_flow(roots[0], visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, counter)

# 3. Exportar
if all_data:
    df = pd.DataFrame(all_data)
    cols = ['Proceso', 'Orden', 'Rol (Responsable)', 'Actividad', 'Descripción', 'Tipo', 'ID']
    df = df[cols]
    output_file = 'flujos_final_corregido.csv'
    df.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')
    print("-" * 30)
    print(f"¡Listo! Se generó: {output_file}")
    # Muestra para verificar
    print(df[df['Tipo'] == 'Tarea'][['Rol (Responsable)', 'Actividad']].head(10))
else:
    print("No se encontraron datos.")

Recursos identificados en el archivo: 56
Ejemplos de recursos: [('ProfesionaldeNomina', 'Profesional de Nómina'), ('CoordinadorTalentohumano', 'Coordinador Talento humano'), ('CoordinadordeCostos', 'Coordinador de Costos')]
Procesando flujo: Proceso principal...
Procesando flujo: Trámites y permisos ante el Ministerio de Defensa...
------------------------------
¡Listo! Se generó: flujos_final_corregido.csv
                                    Rol (Responsable)  \
3                         Técnico de Talento Humano I   
5                           Técnico de Talento Humano   
7   Asistente Vicepresidencia Administrativa y Ope...   
12  Vicepresidente Administrativo en Entidad Desce...   
17                          Secretaria de Presidencia   
19  Presidente de Entidad Descentralizada del Sect...   
22                          Secretaria de Presidencia   
32                        Técnico de Talento Humano I   
34                         Coordinador Talento humano   
38                 

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

# --- FUNCIÓN PARA LIMPIAR HTML ---
def clean_html(raw_html):
    if not raw_html: return ""
    text = html.unescape(raw_html)
    text = re.sub(r'<[^>]+>', ' ', text)
    return " ".join(text.split())

# 1. Carga del Archivo
xml_file = "viaticos.bpmn"
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró '{xml_file}'")
    exit()

# Namespaces genéricos
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

# --- MAPA DE RECURSOS (ID -> NOMBRE) ---
resource_map = {}
for elem in root.iter():
    if elem.tag.endswith('resource'):
        r_id = elem.get('id')
        r_name = elem.get('name')
        if r_id and r_name:
            resource_map[r_id] = r_name

print(f"Recursos identificados: {len(resource_map)}")

# Lista final
all_data = []

# --- FUNCIÓN RECURSIVA (DFS) ---
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id):
    if node_id in visited: return order_counter
    visited.add(node_id)
    
    # Datos básicos
    name = elements.get(node_id, '')
    e_type = element_types.get(node_id, 'Desconocido')
    clean_desc = clean_html(descriptions.get(node_id, ''))
    
    # --- ROL ---
    role = explicit_roles.get(node_id)
    if not role: role = lanes.get(node_id)
    if not role: role = 'Sistema / General'

    # Guardar datos
    if name or e_type in ['Inicio', 'Fin', 'Compuerta']:
        clean_name = name.replace('\n', ' ').strip()
        all_data.append({
            'Proceso Principal': process_name,
            'Sub-Flujo (ID Inicio)': flow_group_id, # Para diferenciar las islas
            'Orden': order_counter,
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Siguientes pasos
    for next_node in graph.get(node_id, []):
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id)
    
    return order_counter

# 2. PROCESAMIENTO GLOBAL
processes = [p for p in root.iter() if p.tag.endswith('process')]

for process in processes:
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Analizando contenedor: {process_name}...")

    # Estructuras de datos para ESTE proceso
    elements = {}
    element_types = {}
    lanes = {}
    descriptions = {}
    explicit_roles = {}
    graph = defaultdict(list)
    
    # A. Lanes
    for lane in process.iter():
        if lane.tag.endswith('lane'):
            lane_name = lane.get('name')
            for ref in lane.iter():
                if ref.tag.endswith('flowNodeRef') and ref.text:
                    lanes[ref.text.strip()] = lane_name

    # B. Nodos y Roles
    for node in process.iter():
        tag_clean = node.tag.split('}')[-1]
        nid = node.get('id')
        if not nid: continue

        if 'Task' in tag_clean: e_type = 'Tarea'
        elif 'Gateway' in tag_clean: e_type = 'Compuerta'
        elif 'startEvent' in tag_clean: e_type = 'Inicio'
        elif 'endEvent' in tag_clean: e_type = 'Fin'
        elif 'Event' in tag_clean: e_type = 'Evento'
        else: continue 

        elements[nid] = node.get('name', '')
        element_types[nid] = e_type
        
        # Doc
        for child in node:
            if child.tag.endswith('documentation') and child.text:
                descriptions[nid] = child.text

        # Rol Explícito
        if 'Task' in tag_clean:
            found_role = None
            for subchild in node.iter():
                if subchild.text:
                    candidate_id = subchild.text.strip().split(':')[-1]
                    if candidate_id in resource_map:
                        found_role = resource_map[candidate_id]
                        break
            if found_role: explicit_roles[nid] = found_role

    # C. Conexiones
    for seq in process.iter():
        if seq.tag.endswith('sequenceFlow'):
            s = seq.get('sourceRef')
            t = seq.get('targetRef')
            if s and t: graph[s].append(t)

    # --- D. LÓGICA DE COMPONENTES DISCONEXAS (ISLAS) ---
    visited = set()
    global_counter = 1
    
    # 1. Encontrar TODOS los nodos de inicio
    start_nodes = [n for n, t in element_types.items() if t == 'Inicio']
    
    print(f"  > Se encontraron {len(start_nodes)} puntos de inicio en este diagrama.")

    # Iterar sobre CADA inicio encontrado
    for start_node in start_nodes:
        if start_node not in visited:
            # Usamos el nombre del evento de inicio para identificar el sub-flujo
            start_name = elements.get(start_node, 'Flujo Sin Nombre')
            print(f"    -> Recorriendo flujo iniciado en: {start_name}")
            
            # Reseteamos el contador para cada flujo independiente (opcional, o podrías dejarlo continuo)
            flow_counter = 1 
            traverse_flow(start_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, flow_counter, start_name)

    # 2. Barrido final: ¿Quedó algo sin visitar? (Islas sin evento de inicio formal)
    all_ids = set(elements.keys())
    unvisited = all_ids - visited
    
    while unvisited:
        # Buscar un nodo raíz dentro de lo que queda
        # (Nodos que no son destino de nadie dentro del grupo de no visitados)
        sub_graph_targets = {t for src, tgts in graph.items() if src in unvisited for t in tgts}
        roots = list(unvisited - sub_graph_targets)
        
        if not roots:
            # Si es un ciclo cerrado, forzamos la entrada por cualquiera
            root_node = list(unvisited)[0]
        else:
            root_node = roots[0]
            
        print(f"    -> Detectado flujo huérfano (sin inicio formal) empezando en: {elements.get(root_node, root_node)}")
        traverse_flow(root_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, "Flujo Huérfano")
        
        # Recalcular no visitados
        unvisited = all_ids - visited

# 3. Exportar
if all_data:
    df = pd.DataFrame(all_data)
    cols = ['Proceso Principal', 'Sub-Flujo (ID Inicio)', 'Orden', 'Rol (Responsable)', 'Actividad', 'Descripción', 'Tipo', 'ID']
    df = df[cols]
    output_file = 'flujos_completos_multiproceso.csv'
    df.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')
    print("-" * 30)
    print(f"¡Listo! Se capturaron {len(df)} actividades totales.")
    print(f"Se generó: {output_file}")
    print(df[['Proceso Principal', 'Sub-Flujo (ID Inicio)', 'Actividad']].head(10))
else:
    print("No se encontraron datos.")

Recursos identificados: 56
Analizando contenedor: Proceso principal...
  > Se encontraron 0 puntos de inicio en este diagrama.
Analizando contenedor: Trámites y permisos ante el Ministerio de Defensa...
  > Se encontraron 1 puntos de inicio en este diagrama.
    -> Recorriendo flujo iniciado en: 
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 1
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 6
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 2
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 6
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 1
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 3
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 4
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 2
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 7
    -> Detectado flujo huérfano (sin inicio formal) empezando en: 2
    -> Detectado flujo

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

# --- FUNCIÓN PARA LIMPIAR HTML ---
def clean_html(raw_html):
    if not raw_html: return ""
    text = html.unescape(raw_html)
    text = re.sub(r'<[^>]+>', ' ', text)
    return " ".join(text.split())

# 1. CARGA DEL ARCHIVO
xml_file = "viaticos.bpmn" # Asegúrate de que el nombre sea correcto
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{xml_file}'")
    exit()

# Namespaces genéricos
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

# --- MAPA DE RECURSOS (ID -> NOMBRE REAL) ---
resource_map = {}
for elem in root.iter():
    if elem.tag.endswith('resource'):
        r_id = elem.get('id')
        r_name = elem.get('name')
        if r_id and r_name:
            resource_map[r_id] = r_name

print(f"Recursos identificados en el sistema: {len(resource_map)}")

all_data = []

# --- FUNCIÓN RECURSIVA (RECORRIDO DEL FLUJO) ---
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id):
    if node_id in visited: return order_counter
    visited.add(node_id)
    
    # Datos del nodo actual
    name = elements.get(node_id, '')
    e_type = element_types.get(node_id, 'Desconocido')
    clean_desc = clean_html(descriptions.get(node_id, ''))
    
    # Determinación del ROL (Prioridad: Explícito > Carril > Defecto)
    role = explicit_roles.get(node_id)
    if not role: role = lanes.get(node_id)
    if not role: role = 'Sistema / General'

    # Guardamos TODO en la lista temporal (luego filtraremos)
    # Es importante guardar todo aquí para mantener la lógica de rastreo
    if name or e_type in ['Inicio', 'Fin', 'Compuerta', 'Tarea']:
        clean_name = name.replace('\n', ' ').strip()
        all_data.append({
            'Proceso Principal': process_name,
            'Sub-Flujo': flow_group_id,
            'Orden_Flujo': order_counter, # Orden técnico en el grafo
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Avanzar a los siguientes nodos
    for next_node in graph.get(node_id, []):
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id)
    
    return order_counter

# 2. PROCESAMIENTO DE PROCESOS
processes = [p for p in root.iter() if p.tag.endswith('process')]

for process in processes:
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Analizando: {process_name}...")

    # Estructuras locales
    elements = {}
    element_types = {}
    lanes = {}
    descriptions = {}
    explicit_roles = {}
    graph = defaultdict(list)
    
    # A. Lanes (Carriles visuales)
    for lane in process.iter():
        if lane.tag.endswith('lane'):
            lane_name = lane.get('name')
            for ref in lane.iter():
                if ref.tag.endswith('flowNodeRef') and ref.text:
                    lanes[ref.text.strip()] = lane_name

    # B. Nodos (Tareas, Compuertas, Eventos)
    for node in process.iter():
        tag_clean = node.tag.split('}')[-1]
        nid = node.get('id')
        if not nid: continue

        # Clasificación
        if 'Task' in tag_clean: e_type = 'Tarea'
        elif 'Gateway' in tag_clean: e_type = 'Compuerta'
        elif 'startEvent' in tag_clean: e_type = 'Inicio'
        elif 'endEvent' in tag_clean: e_type = 'Fin'
        elif 'Event' in tag_clean: e_type = 'Evento'
        else: continue 

        elements[nid] = node.get('name', '')
        element_types[nid] = e_type
        
        # Descripción
        for child in node:
            if child.tag.endswith('documentation') and child.text:
                descriptions[nid] = child.text

        # Rol Explícito (Busqueda profunda de resourceRef)
        if 'Task' in tag_clean:
            found_role = None
            for subchild in node.iter():
                if subchild.text:
                    # Limpieza de prefijos (tns:Recurso -> Recurso)
                    candidate_id = subchild.text.strip().split(':')[-1]
                    if candidate_id in resource_map:
                        found_role = resource_map[candidate_id]
                        break
            if found_role: explicit_roles[nid] = found_role

    # C. Conexiones
    for seq in process.iter():
        if seq.tag.endswith('sequenceFlow'):
            s = seq.get('sourceRef')
            t = seq.get('targetRef')
            if s and t: graph[s].append(t)

    # D. Ejecución del Recorrido (Múltiples inicios + Huérfanos)
    visited = set()
    
    # 1. Inicios Formales
    start_nodes = [n for n, t in element_types.items() if t == 'Inicio']
    for start_node in start_nodes:
        if start_node not in visited:
            start_name = elements.get(start_node, 'Flujo Principal')
            traverse_flow(start_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, start_name)

    # 2. Huérfanos (Islas sin inicio)
    all_ids = set(elements.keys())
    unvisited = all_ids - visited
    while unvisited:
        sub_graph_targets = {t for src, tgts in graph.items() if src in unvisited for t in tgts}
        roots = list(unvisited - sub_graph_targets)
        root_node = roots[0] if roots else list(unvisited)[0]
        
        traverse_flow(root_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, "Flujo Huérfano/Desconectado")
        unvisited = all_ids - visited

# 3. EXPORTACIÓN FILTRADA
if all_data:
    df = pd.DataFrame(all_data)
    
    # --- FILTRO FINAL: SOLO TAREAS ---
    # Aquí es donde eliminamos compuertas y eventos del reporte final
    df_tareas = df[df['Tipo'] == 'Tarea'].copy()
    
    # Opcional: Crear un consecutivo limpio para el reporte (1, 2, 3...) 
    # ignorando los saltos que dejan las compuertas.
    # Si prefieres ver el "salto" numérico del diagrama, borra la línea de abajo.
    df_tareas['Orden_Reporte'] = range(1, len(df_tareas) + 1)

    # Selección de columnas
    cols = ['Proceso Principal', 'Sub-Flujo', 'Orden_Reporte', 'Rol (Responsable)', 'Actividad', 'Descripción', 'Tipo']
    df_tareas = df_tareas[cols]
    
    output_file = 'tareas_finales.csv'
    df_tareas.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')
    
    print("-" * 30)
    print(f"¡Éxito! Archivo generado: {output_file}")
    print(f"Total de tareas extraídas: {len(df_tareas)}")
    print("Muestra de datos:")
    print(df_tareas[['Sub-Flujo', 'Rol (Responsable)', 'Actividad']].head(10))
else:
    print("No se encontró información para exportar.")

Recursos identificados en el sistema: 56
Analizando: Proceso principal...
Analizando: Trámites y permisos ante el Ministerio de Defensa...
------------------------------
¡Éxito! Archivo generado: tareas_finales.csv
Total de tareas extraídas: 23
Muestra de datos:
   Sub-Flujo                                  Rol (Responsable)  \
3                                   Técnico de Talento Humano I   
5                                     Técnico de Talento Humano   
7             Asistente Vicepresidencia Administrativa y Ope...   
12            Vicepresidente Administrativo en Entidad Desce...   
17                                    Secretaria de Presidencia   
19            Presidente de Entidad Descentralizada del Sect...   
22                                    Secretaria de Presidencia   
32                                  Técnico de Talento Humano I   
34                                   Coordinador Talento humano   
38                                    Técnico de Talento Humano   


# Prueba con Bizagi

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

In [4]:
def clean_html(raw_html):
    if not raw_html: return ""
    text = html.unescape(raw_html)
    text = re.sub(r'<[^>]+>', ' ', text)
    return " ".join(text.split())

In [5]:
xml_file = "viaticos.bpmn" 
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{xml_file}'")
    exit()

In [6]:
# --- Namespaces genéricos ---
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

In [7]:
# --- Mapa de recursos ---
resource_map = {}
for elem in root.iter():
    if elem.tag.endswith('resource'):
        r_id = elem.get('id')
        r_name = elem.get('name')
        if r_id and r_name:
            resource_map[r_id] = r_name

print(f"Recursos identificados: {len(resource_map)}")

Recursos identificados: 56


In [8]:
# --- funciones recursivas ---
all_data = []
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id):
    if node_id in visited: return order_counter
    visited.add(node_id)
    
    # Datos básicos
    name = elements.get(node_id, '')
    e_type = element_types.get(node_id, 'Desconocido')
    clean_desc = clean_html(descriptions.get(node_id, ''))
    
    # Rol
    role = explicit_roles.get(node_id)
    if not role: role = lanes.get(node_id)
    if not role: role = 'Sistema / General'

    # Guardar en lista temporal
    if name or e_type in ['Inicio', 'Fin', 'Compuerta', 'Tarea']:
        clean_name = name.replace('\n', ' ').strip()
        all_data.append({
            'Proceso Principal': process_name,
            'Sub-Flujo': flow_group_id, # Se guarda internamente por si acaso, pero no se exportará
            'Orden_Tecnico': order_counter,
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Siguientes pasos
    for next_node in graph.get(node_id, []):
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id)
    
    return order_counter

In [9]:
# --- procesamiento ---
processes = [p for p in root.iter() if p.tag.endswith('process')]

for process in processes:
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Analizando: {process_name}...")

    elements = {}
    element_types = {}
    lanes = {}
    descriptions = {}
    explicit_roles = {}
    graph = defaultdict(list)
    
    # A. Lanes
    for lane in process.iter():
        if lane.tag.endswith('lane'):
            lane_name = lane.get('name')
            for ref in lane.iter():
                if ref.tag.endswith('flowNodeRef') and ref.text:
                    lanes[ref.text.strip()] = lane_name

    # B. Nodos
    for node in process.iter():
        tag_clean = node.tag.split('}')[-1]
        nid = node.get('id')
        if not nid: continue

        if 'Task' in tag_clean: e_type = 'Tarea'
        elif 'Gateway' in tag_clean: e_type = 'Compuerta'
        elif 'startEvent' in tag_clean: e_type = 'Inicio'
        elif 'endEvent' in tag_clean: e_type = 'Fin'
        elif 'Event' in tag_clean: e_type = 'Evento'
        else: continue 

        elements[nid] = node.get('name', '')
        element_types[nid] = e_type
        
        # Descripción
        for child in node:
            if child.tag.endswith('documentation') and child.text:
                descriptions[nid] = child.text

        # Rol Explícito
        if 'Task' in tag_clean:
            found_role = None
            for subchild in node.iter():
                if subchild.text:
                    candidate_id = subchild.text.strip().split(':')[-1]
                    if candidate_id in resource_map:
                        found_role = resource_map[candidate_id]
                        break
            if found_role: explicit_roles[nid] = found_role

    # C. Conexiones
    for seq in process.iter():
        if seq.tag.endswith('sequenceFlow'):
            s = seq.get('sourceRef')
            t = seq.get('targetRef')
            if s and t: graph[s].append(t)

    # D. Ejecución DFS
    visited = set()
    start_nodes = [n for n, t in element_types.items() if t == 'Inicio']
    
    for start_node in start_nodes:
        if start_node not in visited:
            start_name = elements.get(start_node, 'Flujo Principal')
            traverse_flow(start_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, start_name)

    # Huérfanos
    all_ids = set(elements.keys())
    unvisited = all_ids - visited
    while unvisited:
        sub_graph_targets = {t for src, tgts in graph.items() if src in unvisited for t in tgts}
        roots = list(unvisited - sub_graph_targets)
        root_node = roots[0] if roots else list(unvisited)[0]
        traverse_flow(root_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, "Huérfano")
        unvisited = all_ids - visited

Analizando: Proceso principal...
Analizando: Trámites y permisos ante el Ministerio de Defensa...


In [10]:
# --- exportacion final ---
if all_data:
    df = pd.DataFrame(all_data)
    
    # Filtrar SOLO tareas
    df_tareas = df[df['Tipo'] == 'Tarea'].copy()
    
    # Crear consecutivo limpio (1, 2, 3...)
    df_tareas['Orden_Reporte'] = range(1, len(df_tareas) + 1)

    # --- DEFINIR COLUMNAS FINALES ---
    # Orden solicitado: Orden, ID, Actividad, Descripción, Rol, Proceso Principal
    cols_finales = [
        'Orden_Reporte', 
        'ID', 
        'Actividad', 
        'Descripción', 
        'Rol (Responsable)', 
        'Proceso Principal'
    ]
    
    df_tareas = df_tareas[cols_finales]
    
    output_file = 'tareas_reporte_final.csv'
    df_tareas.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')
    
    print("-" * 30)
    print(f"¡Listo! Archivo generado: {output_file}")
    print("Primeras filas:")
    print(df_tareas.head())
else:
    print("No se encontraron datos.")

------------------------------
¡Listo! Archivo generado: tareas_reporte_final.csv
Primeras filas:
    Orden_Reporte                                       ID  \
3               1  Id_fb8630e7-d1da-490d-9693-28e7f5e9ad7b   
5               2  Id_526e933d-ecee-4aef-bdb0-90fae0f44a23   
7               3  Id_4f66bc21-38ad-40b8-a34a-87d391fb9ed1   
12              4  Id_bbd01d87-d095-4aab-afd1-10e2716047c6   
17              5  Id_9914f295-cea1-4539-8f2b-0e9b19d3eebe   

                                 Actividad  \
3        Recibir Certificación de tiquetes   
5   Generar Resolución de comisión y otros   
7                       Revisar documentos   
12                   Aprobar Documentación   
17            Revisar documentos de salida   

                                          Descripción  \
3   Recibir por medio de correo electrónico el doc...   
5   Proyectar la resolución de comisión en formato...   
7   Revisar que la información contenida en los do...   
12  Garantizar que la do

# Algoritmo 2

In [12]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import re
import html

def clean_html(raw_html):
    if not raw_html: return ""
    text = html.unescape(raw_html)
    text = re.sub(r'<[^>]+>', ' ', text)
    return " ".join(text.split())

xml_file = "viaticos.bpmn" 
try:
    tree = ET.parse(xml_file)
    root = tree.getroot()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{xml_file}'")
    exit()

# --- Namespaces genéricos ---
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

# --- Mapa de recursos ---
resource_map = {}
for elem in root.iter():
    if elem.tag.endswith('resource'):
        r_id = elem.get('id')
        r_name = elem.get('name')
        if r_id and r_name:
            resource_map[r_id] = r_name

print(f"Recursos identificados: {len(resource_map)}")

# --- funciones recursivas ---
all_data = []
def traverse_flow(node_id, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id):
    if node_id in visited: return order_counter
    visited.add(node_id)
    
    # Datos básicos
    name = elements.get(node_id, '')
    e_type = element_types.get(node_id, 'Desconocido')
    clean_desc = clean_html(descriptions.get(node_id, ''))
    
    # Rol
    role = explicit_roles.get(node_id)
    if not role: role = lanes.get(node_id)
    if not role: role = 'Sistema / General'

    # Guardar en lista temporal
    if name or e_type in ['Inicio', 'Fin', 'Compuerta', 'Tarea']:
        clean_name = name.replace('\n', ' ').strip()
        all_data.append({
            'Proceso Principal': process_name,
            'Sub-Flujo': flow_group_id,
            'Orden_Tecnico': order_counter,
            'Rol (Responsable)': role,
            'Actividad': clean_name,
            'Descripción': clean_desc,
            'Tipo': e_type,
            'ID': node_id
        })
        order_counter += 1

    # Siguientes pasos
    for next_node in graph.get(node_id, []):
        order_counter = traverse_flow(next_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, order_counter, flow_group_id)
    
    return order_counter

# --- procesamiento ---
processes = [p for p in root.iter() if p.tag.endswith('process')]

for process in processes:
    process_name = process.get('name', 'Proceso Sin Nombre')
    print(f"Analizando: {process_name}...")

    elements = {}
    element_types = {}
    node_raw_tags = {} 
    lanes = {}
    descriptions = {}
    explicit_roles = {}
    graph = defaultdict(list)
    
    # A. Lanes
    for lane in process.iter():
        if lane.tag.endswith('lane'):
            lane_name = lane.get('name')
            for ref in lane.iter():
                if ref.tag.endswith('flowNodeRef') and ref.text:
                    lanes[ref.text.strip()] = lane_name

    # B. Nodos
    for node in process.iter():
        tag_clean = node.tag.split('}')[-1]
        nid = node.get('id')
        if not nid: continue
        
        node_raw_tags[nid] = tag_clean

        if 'Task' in tag_clean: e_type = 'Tarea'
        elif 'Gateway' in tag_clean: e_type = 'Compuerta'
        elif 'startEvent' in tag_clean: e_type = 'Inicio'
        elif 'endEvent' in tag_clean: e_type = 'Fin'
        elif 'Event' in tag_clean: e_type = 'Evento'
        else: continue 

        elements[nid] = node.get('name', '')
        element_types[nid] = e_type
        
        # Descripción
        for child in node:
            if child.tag.endswith('documentation') and child.text:
                descriptions[nid] = child.text

        # Rol Explícito
        if 'Task' in tag_clean:
            found_role = None
            for subchild in node.iter():
                if subchild.text:
                    candidate_id = subchild.text.strip().split(':')[-1]
                    if candidate_id in resource_map:
                        found_role = resource_map[candidate_id]
                        break
            if found_role: explicit_roles[nid] = found_role

    # C. Conexiones
    for seq in process.iter():
        if seq.tag.endswith('sequenceFlow'):
            s = seq.get('sourceRef')
            t = seq.get('targetRef')
            if s and t: graph[s].append(t)

    # --- PASO EXTRA: Desconexión Inteligente y Registro de Puntos de Reinicio ---
    
    # 1. Calcular "In-Degree"
    incoming_count = defaultdict(int)
    for src, targets in graph.items():
        for t in targets:
            incoming_count[t] += 1
            
    # 2. Identificar Gateways
    gateways_to_isolate = set()
    for src_id, targets in graph.items():
        src_tag = node_raw_tags.get(src_id, '')
        if 'intermediateCatchEvent' in src_tag:
            for target_id in targets:
                target_tag = node_raw_tags.get(target_id, '')
                # Si es Gateway Convergente (>1 entrada)
                if 'Gateway' in target_tag and incoming_count[target_id] > 1:
                    gateways_to_isolate.add(target_id)

    # 3. Aplicar corte y GUARDAR PRIORIDAD
    # Creamos una lista para saber EXACTAMENTE dónde continuar después
    priority_restart_nodes = []

    for g_id in gateways_to_isolate:
        if g_id in graph:
            removed_targets = graph[g_id]
            # Guardamos estos nodos como "VIP" para reiniciar el DFS
            priority_restart_nodes.extend(removed_targets)
            
            # Cortamos la conexión
            graph[g_id] = [] 
            print(f"   [Corte Virtual] Gateway {g_id}: Se desconectó el flujo hacia {removed_targets}. Se agendaron para reinicio inmediato.")

    # D. Ejecución DFS
    visited = set()
    start_nodes = [n for n, t in element_types.items() if t == 'Inicio']
    
    # 1. Flujo Principal (Start Events)
    for start_node in start_nodes:
        if start_node not in visited:
            start_name = elements.get(start_node, 'Flujo Principal')
            traverse_flow(start_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, start_name)

    # 2. Puntos de Reinicio Prioritarios (Lo que estaba justo después del Gateway)
    # Esto garantiza que el nodo siguiente al gateway sea el primero del nuevo bloque
    for p_node in priority_restart_nodes:
        if p_node not in visited:
            # Usamos un nombre distintivo para el subflujo
            p_name = elements.get(p_node, "Fase Posterior al Cierre")
            traverse_flow(p_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, "Continuación Post-Gateway")

    # 3. Otros Huérfanos (Limpieza general por si quedaron islas sueltas no relacionadas)
    all_ids = set(elements.keys())
    unvisited = all_ids - visited
    while unvisited:
        sub_graph_targets = {t for src, tgts in graph.items() if src in unvisited for t in tgts}
        roots = list(unvisited - sub_graph_targets)
        root_node = roots[0] if roots else list(unvisited)[0]
        
        traverse_flow(root_node, visited, graph, elements, element_types, lanes, descriptions, explicit_roles, process_name, 1, "Otros Flujos")
        unvisited = all_ids - visited

# --- exportacion final ---
if all_data:
    df = pd.DataFrame(all_data)
    
    # Filtrar SOLO tareas
    df_tareas = df[df['Tipo'] == 'Tarea'].copy()
    
    # Crear consecutivo limpio
    df_tareas['Orden_Reporte'] = range(1, len(df_tareas) + 1)

    cols_finales = [
        'Orden_Reporte', 
        'ID', 
        'Actividad', 
        'Descripción', 
        'Rol (Responsable)', 
        'Proceso Principal'
    ]
    
    df_tareas = df_tareas[cols_finales]
    
    output_file = 'tareas_reporte_final_throw_catch.csv'
    df_tareas.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')
    
    print("-" * 30)
    print(f"¡Listo! Archivo generado: {output_file}")
    print("Primeras filas:")
    print(df_tareas.head())
else:
    print("No se encontraron datos.")

Recursos identificados: 56
Analizando: Proceso principal...
Analizando: Trámites y permisos ante el Ministerio de Defensa...
   [Corte Virtual] Gateway Id_cd5d3e5f-ea5d-456b-89dc-24274baeca2f: Se desconectó el flujo hacia ['Id_4dd47aee-674a-4987-8c31-0d016ca8ac19']. Se agendaron para reinicio inmediato.
------------------------------
¡Listo! Archivo generado: tareas_reporte_final_throw_catch.csv
Primeras filas:
    Orden_Reporte                                       ID  \
3               1  Id_fb8630e7-d1da-490d-9693-28e7f5e9ad7b   
5               2  Id_526e933d-ecee-4aef-bdb0-90fae0f44a23   
7               3  Id_4f66bc21-38ad-40b8-a34a-87d391fb9ed1   
12              4  Id_bbd01d87-d095-4aab-afd1-10e2716047c6   
17              5  Id_9914f295-cea1-4539-8f2b-0e9b19d3eebe   

                                 Actividad  \
3        Recibir Certificación de tiquetes   
5   Generar Resolución de comisión y otros   
7                       Revisar documentos   
12                   Aprobar